In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import cv2
import torch
from fastai2.vision.learner import create_cnn_model

from torchvision.models import densenet121

/home/user/Dev/plants-image-analysis/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# https://www.kaggle.com/code/lextoumbourou/plant-pathology-2020-eda-training-fastai2
# Download model with `./scripts/download_fastai_model.sh`

PROJECT_ROOT = os.path.abspath(os.path.join(os.curdir, os.pardir, os.pardir))
MODEL_FPATH = os.path.join(PROJECT_ROOT, "models", "fastai_model.pth")
SAMPLE_FPATH = os.path.join(PROJECT_ROOT, "data", "rust_multiple_diseases_sample.png")

In [4]:
checkpoint_data = torch.load(MODEL_FPATH)

In [46]:
# cut=-1 - from meta
model = create_cnn_model(densenet121, n_out=4, cut=-1)
model.load_state_dict(checkpoint_data["model"])
model = model.eval()

model

Sequential(
  (0): Sequential(
    (0): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine

In [47]:
# Read and resize
img = cv2.imread(SAMPLE_FPATH)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img_size = (512, 512)
img = cv2.resize(img, img_size, interpolation=cv2.INTER_LINEAR)

In [48]:
# Prepare tensor 
img_t = torch.from_numpy(img)
img_t = img_t.permute(2, 0, 1)
img_t = img_t.unsqueeze(0)
img_t = img_t.float() / 255
# img_t = img_t.half()

In [58]:
with torch.no_grad():
    pred_t = model(img_t)

pred_norm_t = torch.softmax(pred_t, axis=-1)


In [59]:
labels = ['healthy', 'multiple_diseases', 'rust', 'scab']

pred_t, pred_norm_t
# Norm ~ Normalize probabilities of classes

(tensor([[ 0.3169,  0.2219,  2.0037, -0.6585]]),
 tensor([[0.1301, 0.1183, 0.7026, 0.0490]]))